In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

from glob import glob
from tqdm import tqdm
from collections import deque

import requests
import pandas as pd
import numpy as np
import os

In [2]:
gp_path = f"{os.getcwd().split('web_crawling')[0]}web_crawling"
print(gp_path)

c:\Users\NT550-045\Desktop\ml2\web_crawling


In [3]:
steampy_path = glob(f'{gp_path}\\**\\steampy\\appid_*.csv', recursive=True)[::2]
user_peak = glob(f'{gp_path}\\**\\MPW\\**\\*_user.csv', recursive=True)[:5]
review_path = glob(f'{gp_path}\\**\\MPW\\**\\*_review.csv', recursive=True)[:5]
steam_store_path = glob(f'{gp_path}\\**\\CGW\\**\\final\\*.csv', recursive=True)
store_date_path = glob(f'{gp_path}\\**\\CGW\\**\\csvs\\*_date.csv', recursive=True)

In [4]:
print(steampy_path,steam_store_path,user_peak,review_path, store_date_path, sep='\n\n')

['c:\\Users\\NT550-045\\Desktop\\ml2\\web_crawling\\LSH\\steampy\\appid_2018.csv', 'c:\\Users\\NT550-045\\Desktop\\ml2\\web_crawling\\LSH\\steampy\\appid_2019.csv', 'c:\\Users\\NT550-045\\Desktop\\ml2\\web_crawling\\LSH\\steampy\\appid_2020.csv', 'c:\\Users\\NT550-045\\Desktop\\ml2\\web_crawling\\LSH\\steampy\\appid_2021.csv', 'c:\\Users\\NT550-045\\Desktop\\ml2\\web_crawling\\LSH\\steampy\\appid_2022.csv']

['c:\\Users\\NT550-045\\Desktop\\ml2\\web_crawling\\CGW\\data_handling\\final\\steam_store_after_pcs_2018.csv', 'c:\\Users\\NT550-045\\Desktop\\ml2\\web_crawling\\CGW\\data_handling\\final\\steam_store_after_pcs_2019.csv', 'c:\\Users\\NT550-045\\Desktop\\ml2\\web_crawling\\CGW\\data_handling\\final\\steam_store_after_pcs_2020.csv', 'c:\\Users\\NT550-045\\Desktop\\ml2\\web_crawling\\CGW\\data_handling\\final\\steam_store_after_pcs_2021.csv', 'c:\\Users\\NT550-045\\Desktop\\ml2\\web_crawling\\CGW\\data_handling\\final\\steam_store_after_pcs_2022.csv']

['c:\\Users\\NT550-045\\Desktop

```
# 스팀스파이 파일
t_spy = pd.read_csv(steampy_path[0], sep='\t')

# 스팀차트 파일
# 불필요한 컬럼 제거 ; Unamed 0 : 0 / usecols 옵션사용
t_peak = pd.read_csv(user_peak[0], sep=';', usecols=['App_id', '24_Hour_Peak', 'All_time_peak'])
# App_id 컬럼 이름 변경
t_peak.rename(columns={'App_id':'appid'},inplace=True)

# 스팀 타켓값 파일
# 불필요한 컬럼 제거 ; Unamed 0 : 0 / usecols 옵션사용
t_review = pd.read_csv(review_path[0], sep=';', usecols=['App_id', 'Review'])
# App_id 컬럼 이름 변경
t_review.rename(columns={'App_id':'appid'},inplace=True)

# 스팀 스토어 파일
t_store = pd.read_csv(steam_store_path[0], sep=';')
# 불필요한 컬럼 제거 ; Unamed 0 : 0 / 컬럼 인덱싱
t_store = t_store[t_store.columns[1:]]
```

In [121]:
spy_del = ['developer', 'publisher', 'tags', 'languages']

review_set = set()
merge_df = pd.DataFrame()

for i in range(5):
    t_spy = pd.read_csv(steampy_path[i], sep='\t')
    t_spy.drop(spy_del, axis=1, inplace=True)
    
    t_peak = pd.read_csv(user_peak[i], sep=';', usecols=['App_id', '24_Hour_Peak', 'All_time_peak'])
    t_peak.rename(columns={'App_id':'appid'},inplace=True)

    t_review = pd.read_csv(review_path[i], sep=';', usecols=['App_id', 'Review'])
    t_review.rename(columns={'App_id':'appid'},inplace=True)

    t_store = pd.read_csv(steam_store_path[i], sep=';')
    t_store = t_store[t_store.columns[1:-2]]
    
    t_date = pd.read_csv(store_date_path[i])
    
    t_merge = t_peak.merge(t_spy).merge(t_store).merge(t_date).merge(t_review)
    
    # 각 파일마다 타켓의 유니크한 값 확인
    print(t_merge['Review'].unique())
    print(len(t_merge['Review'].unique()))
    print()
    
    # 유니크한 타켓 저장
    review_set |= set(t_merge['Review'].unique())
    
    # 각 연도 파일 합치기
    merge_df = pd.concat([merge_df, t_merge])

['Overwhelmingly Positive' 'Very Positive' 'Positive' 'Mostly Positive'
 'Mixed' 'No user reviews' '8 user reviews' '7 user reviews'
 '4 user reviews' '9 user reviews' '5 user reviews' '6 user reviews'
 '2 user reviews' '1 user reviews' '3 user reviews' 'Mostly Negative'
 'Negative' 'Very Negative']
18

['Overwhelmingly Positive' 'Very Positive' 'Mostly Positive' 'Mixed'
 'Positive' '6 user reviews' 'No user reviews' '4 user reviews'
 '3 user reviews' '8 user reviews' '5 user reviews' '7 user reviews'
 '9 user reviews' '1 user reviews' '2 user reviews' 'Mostly Negative'
 'Very Negative' 'Negative']
18

['Overwhelmingly Positive' 'Very Positive' 'Mixed' 'Mostly Positive'
 'No user reviews' 'Positive' '2 user reviews' '4 user reviews'
 '8 user reviews' '6 user reviews' '7 user reviews' 'None'
 '9 user reviews' '1 user reviews' '5 user reviews' '3 user reviews'
 'Mostly Negative' 'Negative' 'Very Negative']
19

['Overwhelmingly Positive' 'Very Positive' 'Mostly Positive' 'Mixed'
 'Positiv

```
tag 파일 정리
```

In [100]:
tag_path = glob(f'{gp_path}\\**\\steampy\\*tag.csv', recursive=True)
tag_path

['c:\\Users\\reot1\\OneDrive\\바탕 화면\\ml2\\web_crawling\\LSH\\steampy\\appid_2018_tag.csv',
 'c:\\Users\\reot1\\OneDrive\\바탕 화면\\ml2\\web_crawling\\LSH\\steampy\\appid_2019_tag.csv',
 'c:\\Users\\reot1\\OneDrive\\바탕 화면\\ml2\\web_crawling\\LSH\\steampy\\appid_2020_tag.csv',
 'c:\\Users\\reot1\\OneDrive\\바탕 화면\\ml2\\web_crawling\\LSH\\steampy\\appid_2021_tag.csv',
 'c:\\Users\\reot1\\OneDrive\\바탕 화면\\ml2\\web_crawling\\LSH\\steampy\\appid_2022_tag.csv']

In [106]:
tag_merge = pd.DataFrame()
for path in tag_path:
    t_tag = pd.read_csv(path)
    
    # 태그 파일 합치기
    tag_merge = pd.concat([tag_merge, t_tag])

7713
7370
9063
10715
11862


```
데이터 유실 확인

게임 출시일자 추출시 해당 사이트에서 기존에 있던 게임 중 
삭제되거나 문제있는 게임을 목록에서 제거한 것으로 보임.
```

In [120]:
for i in range(5):
    t_spy = pd.read_csv(steampy_path[i], sep='\t')
    t_spy.drop(spy_del, axis=1, inplace=True)
    
    t_peak = pd.read_csv(user_peak[i], sep=';', usecols=['App_id', '24_Hour_Peak', 'All_time_peak'])
    t_peak.rename(columns={'App_id':'appid'},inplace=True)

    t_review = pd.read_csv(review_path[i], sep=';', usecols=['App_id', 'Review'])
    t_review.rename(columns={'App_id':'appid'},inplace=True)

    t_store = pd.read_csv(steam_store_path[i], sep=';')
    t_store = t_store[t_store.columns[1:-2]]
    
    t_date = pd.read_csv(store_date_path[i])
    
    t_merge = t_peak.merge(t_spy).merge(t_store).merge(t_date).merge(t_review)
    print(len(t_merge))
    print(len(t_spy), len(t_peak), len(t_review), len(t_store), len(t_date))
    print()

7635
7713 7713 7713 7713 7639

7306
7370 7370 7370 7370 7313

8979
9063 9063 9063 9063 8985

10600
10715 10715 10715 10715 10606

11732
11862 11862 11862 11862 11745



In [123]:
merge_df['Review'].value_counts()

Very Positive              7498
Positive                   6950
Mixed                      6575
Mostly Positive            4701
1 user reviews             3706
2 user reviews             2908
3 user reviews             2343
4 user reviews             1940
No user reviews            1810
5 user reviews             1669
6 user reviews             1423
7 user reviews             1244
8 user reviews             1100
Mostly Negative             957
9 user reviews              948
Overwhelmingly Positive     270
Negative                    185
Very Negative                17
None                          5
Overwhelmingly Negative       3
Name: Review, dtype: int64

In [132]:
merge_df['Review'].apply(lambda x:'Indifference' if 'user' in x  or 'None' == x else x).apply(lambda x: 'Negative' if 'Negative' in x else x).apply(lambda x: 'Positive' if 'Positive' in x else x ).value_counts(normalize=True)

Positive        0.419852
Indifference    0.412869
Mixed           0.142156
Negative        0.025123
Name: Review, dtype: float64

In [143]:
dataset_path = glob(f'{gp_path}\\**\\merge_dataset', recursive=True)[0]
dataset_path

'c:\\Users\\reot1\\OneDrive\\바탕 화면\\ml2\\web_crawling\\merge_dataset'

In [144]:
merge_df.to_csv(f'{dataset_path}\\raw_merge.csv', sep='\t', index=False)
tag_merge.to_csv(f'{dataset_path}\\tag_merge.csv', index=False)